In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Write training code here and Click "Execute" for a workbench execute job
- Use custom container built in Cloud Build and stored in Artifact Registry
- Cloud Build command: gcloud builds submit --config cloud-build.yaml .
- input and output directory can be /gcs/bucket_name/folder for Cloud Storage path

In [ ]:
MODEL_NAME="runwayml/stable-diffusion-v1-5"
INSTANCE_DIR="/gcs/bucket_name/input_dog"
OUTPUT_DIR="/gcs/bucket_name/dog_lora_output"

! accelerate launch ./diffusers/examples/dreambooth/train_dreambooth_lora.py \
    --pretrained_model_name_or_path=$MODEL_NAME  \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --instance_prompt="a photo of sks dog" \
    --resolution=512 \
    --train_batch_size=1 \
    --use_8bit_adam \
    --mixed_precision="fp16" \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-4 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=500

Convert the lora .bin file to safetensor file, for used in WebUI

In [ ]:
import os;
import re;
import torch;
from safetensors.torch import save_file;

newDict = dict();
checkpoint = torch.load(OUTPUT_DIR + '/pytorch_lora_weights.bin');
for idx, key in enumerate(checkpoint):
    newKey = re.sub('\.processor\.', '_', key);
    newKey = re.sub('mid_block\.', 'mid_block_', newKey);
    newKey = re.sub('_lora.up.', '.lora_up.', newKey);
    newKey = re.sub('_lora.down.', '.lora_down.', newKey);
    newKey = re.sub('\.(\d+)\.', '_\\1_', newKey);
    newKey = re.sub('to_out', 'to_out_0', newKey);
    newKey = 'lora_unet_'+newKey;

    newDict[newKey] = checkpoint[key];

newLoraName = 'pytorch_lora_weights.safetensors';
print("Saving " + newLoraName);
save_file(newDict, OUTPUT_DIR + '/' + newLoraName);

***It's supported to configure NFS using Executor*